# Read Data

1) Preparing the data. 

In [1]:
import glob

In [2]:

def read_text_files(folder_path):
    file_paths = glob.glob(folder_path + '/*.txt')
    texts = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            texts.append(text)
    return texts



In [52]:
# import os

# def read_text_files(folder_path):
#     texts = []
#     for root, _, files in os.walk(folder_path):
#         for file in files:
#             if file.endswith('.txt'):
#                 file_path = os.path.join(root, file)
#                 with open(file_path, 'r', encoding='utf-8') as file:
#                     text = file.read()
#                     texts.append(text)
#     return texts

# Ejemplo de uso

In [3]:

gutenberg_folder_path = '/home/judal/Documentos/CURSOS_2023_I/BigData/tarea_1/Gutenberg_Text/Alcott, Louisa May'
texts = read_text_files(gutenberg_folder_path)
# gutenberg_text = ' '.join(texts)

In [4]:
len(texts)

38

# MapReduce Implementation

2) Writing map and reduce functions.

<!-- 3) Getting a better understanding of how mapper and reducer work. -->

In [5]:
# Importing packages 
import string
import re
import pandas as pd
import itertools

In [6]:
def mapper(chunk):
    word_count = {}
    words = chunk.split()
    for word in words:
        if word not in word_count:
            word_count[word] = 0
        word_count[word] += 1
    return word_count

In [7]:
def reducer(word_counts):
    final_word_count = {}
    for word_count in word_counts:
        for word, count in word_count.items():
            if word not in final_word_count:
                final_word_count[word] = 0
            final_word_count[word] += count
    return final_word_count

In [8]:
data = [
    "Lorem ipsum dolor sit amet",
    "consectetur adipiscing elit",
    "sed do eiusmod tempor incididunt ut labore et dolore magna aliqua"
]

In [9]:
def mapreduce(data, mapper, reducer, num_threads):
    word_counts = []
    for text in data:
        word_count = mapper(text)
        word_counts.append(word_count)
    
    result = reducer(word_counts)
    print(result)
    # reducer()
# # Ejemplo de uso
# num_threads = 4
# word_count_result = mapreduce(gutenberg_text, mapper, reducer, num_threads)
# print(word_count_result)

In [10]:
%%time
mapreduce(texts,mapper,reducer,4)


{'The': 4660, 'Project': 2919, 'Gutenberg': 786, 'EBook': 51, 'of': 46889, 'Under': 40, 'the': 99219, 'Lilacs,': 4, 'by': 7534, 'Louisa': 271, 'May': 467, 'Alcott': 384, 'This': 902, 'eBook': 210, 'is': 10670, 'for': 24461, 'use': 617, 'anyone': 223, 'anywhere': 121, 'at': 11065, 'no': 5076, 'cost': 187, 'and': 89390, 'with': 22509, 'almost': 470, 'restrictions': 75, 'whatsoever.': 74, 'You': 1439, 'may': 1932, 'copy': 344, 'it,': 3097, 'give': 1793, 'it': 15122, 'away': 2212, 'or': 7574, 're-use': 74, 'under': 1102, 'terms': 714, 'License': 298, 'included': 78, 'this': 6729, 'online': 149, 'www.gutenberg.net': 16, 'Title:': 38, 'Lilacs': 2, 'Author:': 38, 'Posting': 11, 'Date:': 49, '19,': 5, '2009': 7, '[EBook': 36, '#3795]': 1, 'Release': 38, 'February,': 6, '2003': 10, 'First': 41, 'Posted:': 3, 'September': 31, '12,': 6, '2001': 11, '[Last': 2, 'updated:': 2, 'April': 50, '1,': 14, '2011]': 1, 'Language:': 38, 'English': 214, 'Character': 38, 'set': 1202, 'encoding:': 38, 'ASCII':

In [34]:
import threading

In [195]:
word_counts = []
final_word_counts = []
lock = threading.Lock()

In [196]:
word_counts

[]

In [197]:
def mapT(chunk,i,j,data):
    
    global word_counts
    
    subdata = data[i:j]

    for text in subdata:
        word_count = {}
        words = text.split()
        for word in words:
            if word not in word_count:
                word_count[word] = 0
            word_count[word] += 1
        with lock:
            word_counts.append(word_count)
    #     final_word_count = {}
    #     for word_count in word_counts:
    #         for word, count in word_count.items():
    #             if word not in final_word_count:
    #                 final_word_count[word] = 0
    #             final_word_count[word] += count
    #     return final_word_count

    
    print("hello",chunk, "->",i," : ",j)

In [238]:
def reduceT(chunk,i,j,word_counts):
    
    global final_word_counts

    subdata = word_counts[i:j]

    # print(len(subdata))

    # for word_count in subdata:
    #     print(chunk,", ",type(word_count)) 


    final_word_count = {}
    for word_count in subdata:    
        for word, count  in word_count.items():
            if word not in final_word_count:
                final_word_count[word] = 0
            final_word_count[word] += count
    with lock:
        final_word_counts.append(final_word_count)
            
    print("hello",chunk, "->",i," : ",j)

In [239]:

def proccessingMap(data, mapT, num_threads):
    threads = []

    k = len(data) // num_threads
    
    for _ in range(num_threads):

        inicio = _ * k
        fin = (_ * k) + k if _ < num_threads - 1 else len(data)

        thread = threading.Thread(target=mapT,args=(_,inicio,fin,data))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()
    
    # # Crear los threads para el mapeo
    # map_threads = []
    # word_counts = []
    #     t = threading.Thread(target=lambda: word_counts.append(mapper(chunk)))
    #     t.start()
    #     map_threads.append(t)

    # # Esperar a que todos los threads de mapeo finalicen
    # for t in map_threads:
    #     t.join()

    # Reducir los resultados
    # result = reducer(word_counts)
    # return result



In [200]:
def proccessingReduce(word_counts, reduceT, num_threads):
    threads = []

    k = len(word_counts) // num_threads
    
    for _ in range(num_threads):

        inicio = _ * k
        fin = (_ * k) + k if _ < num_threads - 1 else len(word_counts)

        thread = threading.Thread(target=reduceT,args=(_,inicio,fin,word_counts))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()


### Map Processing

In [201]:

num_threads = 4
proccessingMap(texts, mapT, num_threads)


hello 0 -> 0  :  9
hello 2 -> 18  :  27
hello 1 -> 9  :  18
hello 3 -> 27  :  38


In [241]:
print(len(texts))
print(len(word_counts))

38
38


In [242]:
print(texts[0][:100])
for word in word_counts:
    print(type(word), end=", ")

The Project Gutenberg EBook of Under the Lilacs, by Louisa May Alcott

This eBook is for the use of 
<class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, 

### Reduce Processing

In [243]:
# Ejemplo de uso
num_threads = 4
proccessingReduce(word_counts, reduceT, num_threads)


hello 0 -> 0  :  9
hello 1 -> 9  :  18
hello 2 -> 18  :  27
hello 3 -> 27  :  38


In [244]:
len(final_word_counts)

4

In [246]:
result = reducer(final_word_counts)

In [247]:
result

{'The': 4660,
 'Project': 2919,
 'Gutenberg': 786,
 'EBook': 51,
 'of': 46889,
 'Under': 40,
 'the': 99219,
 'Lilacs,': 4,
 'by': 7534,
 'Louisa': 271,
 'May': 467,
 'Alcott': 384,
 'This': 902,
 'eBook': 210,
 'is': 10670,
 'for': 24461,
 'use': 617,
 'anyone': 223,
 'anywhere': 121,
 'at': 11065,
 'no': 5076,
 'cost': 187,
 'and': 89390,
 'with': 22509,
 'almost': 470,
 'restrictions': 75,
 'whatsoever.': 74,
 'You': 1439,
 'may': 1932,
 'copy': 344,
 'it,': 3097,
 'give': 1793,
 'it': 15122,
 'away': 2212,
 'or': 7574,
 're-use': 74,
 'under': 1102,
 'terms': 714,
 'License': 298,
 'included': 78,
 'this': 6729,
 'online': 149,
 'www.gutenberg.net': 16,
 'Title:': 38,
 'Lilacs': 2,
 'Author:': 38,
 'Posting': 11,
 'Date:': 49,
 '19,': 5,
 '2009': 7,
 '[EBook': 36,
 '#3795]': 1,
 'Release': 38,
 'February,': 6,
 '2003': 10,
 'First': 41,
 'Posted:': 3,
 'September': 31,
 '12,': 6,
 '2001': 11,
 '[Last': 2,
 'updated:': 2,
 'April': 50,
 '1,': 14,
 '2011]': 1,
 'Language:': 38,
 'Engl

In [248]:
len(result)

99882

## test


In [53]:
import threading

def hello(chunk):
    print("Hello, World!",chunk)

if __name__ == "__main__":
    threads = []
    num_threads = 8

    for _ in range(num_threads):
        thread = threading.Thread(target=hello,args=(_,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

Hello, World! 0
Hello, World! 1
Hello, World! 2
Hello, World! 3
Hello, World! 4
Hello, World! 5
Hello, World! 6
Hello, World! 7
